<a href="https://colab.research.google.com/github/Isha130/Hen-Detection/blob/main/Hen_detection_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FFMPEG installation

In [ ]:
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

Installation finished.


# Making Folders

In [ ]:
%cd /content/
!mkdir Rawvideo
!mkdir finalvideo
%cd Rawvideo

# Checking if the Uploaded video exist or not

In [ ]:
import os

if os.path.exists("/content/Rawvideo/video2.mp4"):
    print("Your video has been successfully uploaded.")
else:
    print("There was an error uploading your video.")


Your video has been successfully uploaded.


# Break video into frames

In [ ]:
import cv2
import os

video = cv2.VideoCapture('/content/Rawvideo/video2.mp4')

if not os.path.exists('Rawvideo'):
    os.makedirs('Rawvideo')

frame_count = 0

while True:

    success, frame = video.read()


    if not success:
        break

    # Write the current frame to a file
    frame_name = f'frame_{frame_count:04d}.jpg'
    frame_path = os.path.join('Rawvideo', frame_name)
    cv2.imwrite(frame_path, frame)

    # Increment the frame counter
    frame_count += 1

# Release the video file
video.release()


In [ ]:
import os

video_name = "Video3.mp4"
video_path = os.path.join("/content/Rawvideo", video_name)

try:
    os.remove(video_path)
    print(f"{video_name} deleted successfully.")
except OSError as e:
    print(f"Error deleting {video_name}: {e}")


# Roboflow installation

In [ ]:
!pip3 install roboflow

#Imports

In [ ]:
from roboflow import Roboflow
import json
from time import sleep
from PIL import Image, ImageDraw
import io
import base64
import requests
from os.path import exists
import os, sys, re, glob

# Predictions

In [ ]:
# workspace code
from roboflow import Roboflow
import json

rf = Roboflow(api_key="5wga3sQ5ayTezA666JQq")
project = rf.workspace().project("hen-ypxio")
dataset = project.version("2")

# grab the model from that project's version
model = dataset.model
print(model)

In [ ]:
import os
from roboflow import Roboflow

rf = Roboflow(api_key="5wga3sQ5ayTezA666JQq")
project = rf.workspace().project("hen-ypxio")
model = project.version("2").model

folder_path = "/content/Rawvideo/"

# loop over all image files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):

        # infer on the current image file
        image_path = os.path.join(folder_path, filename)
        response = model.predict(image_path, confidence=40, overlap=30)
        predictions = response.json()["predictions"]

        # print the results along with the image filename
        print(f"Image: {filename}")
        print(f"Number of detected hens: {len(predictions)}")
        print(predictions)

# Deleting the original video file

In [ ]:
import os

folder_path = '/content/Rawvideo/'

for filename in os.listdir(folder_path):
    if filename.endswith('.mp4'):
        os.remove(os.path.join(folder_path, filename))


# Drawing Bounding boxes on the extracted frames

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os

# Set the path of the directory containing the images
images_dir = '/content/Rawvideo/'

# Set the path of the directory to save the output images
output_dir = '/content/finalvideo'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through the images in the directory
for image_name in os.listdir(images_dir):
    # Check if the file is a directory or an image file
    if os.path.isdir(os.path.join(images_dir, image_name)):
        continue
    # Get the path of the current image
    image_path = os.path.join(images_dir, image_name)

    # Open the image
    image = Image.open(image_path)

    # Create a new ImageDraw object and load the default font
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    # Loop through the bounding boxes and draw them on the image
    for box in predictions:
        color = "#FFFF00"
        x1 = box['x'] - box['width'] / 2
        x2 = box['x'] + box['width'] / 2
        y1 = box['y'] - box['height'] / 2
        y2 = box['y'] + box['height'] / 2

        draw.rectangle([
            x1, y1, x2, y2
        ], outline=color, width=5)

        if True:
            text = box['class']
            text_size = font.getsize(text)

            # set button size + 10px margins
            button_size = (text_size[0]+20, text_size[1]+20)
            button_img = Image.new('RGBA', button_size, color)
            # put text on button with 10px margins
            button_draw = ImageDraw.Draw(button_img)
            button_draw.text((10, 10), text, font=font, fill=(255,255,255,255))

            # put button on source image in position (0, 0)
            image.paste(button_img, (int(x1), int(y1)))

    # Save the output image in the output directory
    output_path = os.path.join(output_dir, image_name)
    image.save(output_path)


# Combining Frames into A video

In [ ]:
import cv2
import os

# Set the path of the directory containing the extracted frames
frames_dir = '/content/finalvideo'

# Set the output video file name and frame size
output_dir = '/content/finalvideo'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_file = os.path.join(output_dir, 'output_video.mp4')
frame_size = (640, 480)

# Get the list of frame file names in the directory
frame_names = os.listdir(frames_dir)

# Sort the frame file names in ascending order
frame_names.sort()

# Initialize the video writer object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_file, fourcc, 30, frame_size)

# Loop through the frame file names and add them to the video
for frame_name in frame_names:
    frame_path = os.path.join(frames_dir, frame_name)
    frame = cv2.imread(frame_path)

    # Check if the frame is empty before resizing it
    if frame is not None:
        # Resize the frame to match the specified frame size
        frame = cv2.resize(frame, frame_size)

        # Write the frame to the output video file
        video.write(frame)

# Release the video writer object and close the output file
video.release()

Check the number of images in a directory

In [ ]:
import glob

# specify the path to the directory
path = "/content/Rawvideo/*.jpg"

# use glob to get a list of all PNG files in the directory
png_files = glob.glob(path)

# use len to count the number of PNG files
num_png_files = len(png_files)

print(f"There are {num_png_files} PNG files in the directory.")


# Delete junk png in content folder

In [ ]:
import os

folder_path = '/content/Rawvideo/'

for filename in os.listdir(folder_path):
    if filename.endswith('.MOV'):
        os.remove(os.path.join(folder_path, filename))


# Checking directory content

In [ ]:
!ls /content/Rawvideo